In [19]:
import numpy
import pandas as pd
import cloudstorage as gcs
import pickle
from pandas.io.json import json_normalize
import time
import os
from gcloud import storage
from tqdm import tqdm

In [20]:
"""
GCP SDK command lines to check contents, requires SDK and coreutils

gcloud init       #initiate automatic gcloud connection 
gsutil du -s gs://py_spark_ds        #simple count bytes within entire bucket
gsutil ls -lR gs://py_spark_ds | tail -n 1      #summary of total objects and size in entire bucket
gsutil du gs://py_spark_ds > MyTextOutput.txt    #output all file names and sizes to a txt file

# TOTAL: 40501 objects, 39366312918 bytes (36.66 GiB)

"""
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\alanl\\Desktop\\Data Flow Test-45c6beb1f9c3.json'

client = storage.Client()

bucket = client.get_bucket('py_spark_ds')

blobs = bucket.list_blobs()

In [21]:
# read output file with fixed width
user_list = pd.read_fwf('output_list.txt', header = None) 

In [22]:
# extract all user names and put it in a list called list1
list1=[]
for i in user_list[1]:
    list1.append(i.strip("gs://py_spark_ds/"))
len(list1)

40501

In [38]:
22513+1951+7390+8

31862

In [39]:
list3 = list1[31862:]

In [40]:
len(list3)

8639

In [49]:
list3[-1]

'zzzkrak GFYzzz.pkl'

In [42]:
def json_norm(player_list,feature):
    """
    takes the final list containing dictionary and returns parsed json
    :param final_list:
    :param player_name:
    :return: pandas data frame
    """
    df = player_list[feature]
    df = json_normalize(df)
    df.index = df.name
    df = df.drop(['name'],axis=1)
    df_series = df.stack()
    df_normalized = df_series.to_frame().T
    df_normalized.columns = ['_'.join(col).strip() for col in df_normalized.columns.values]
    
    return df_normalized

In [43]:
def convert_int(dataframe):
    """
    convert objects to int
    :param dataframe: 
    :return: 
    """
    for col in dataframe.columns:
        try:
            dataframe[col] = dataframe[col].fillna('999.99')
            dataframe[col] = dataframe[col].astype(int)
        except:
            continue
    return dataframe.loc[:,dataframe.dtypes==int]

In [44]:
def fill_missing(dataframe):
    for col in dataframe.columns:
        dataframe[col] = dataframe[col].fillna(999.99)
    return dataframe

In [46]:
player_info = pd.DataFrame([])
success_count = pd.DataFrame([])
with tqdm(total=8639) as pbar:
    for file in list3:
        s = bucket.get_blob(file)
        try:
            with open("./tmp.pkl", "wb") as file_obj:
                s.download_to_file(file_obj)
        except:
            continue
            
        try:
            with open("./tmp.pkl","rb") as file_obj:
                tmp = pickle.load(file_obj)
        except:
            continue

        weaponCategory = json_norm(tmp,'weaponCategory')
        players = pd.DataFrame([{keys:values for keys,values in tmp['player'].items() if keys in ['name']}])
        stats = json_normalize([{keys:values for (keys,values) in tmp['stats'].items() if keys not in ['modes']}])

        #clean up vehicleCategory to include in dataframe
        vehicleCategory = tmp['vehicleCategory']
        vehicleCategory = json_normalize(vehicleCategory)
        vehicleCategory= vehicleCategory[['extra.kpm','extra.spm','name','stat.destroys','stat.kills','stat.score','stat.time']]
        vehicleCategory.index = vehicleCategory.name
        vehicleCategory = vehicleCategory.drop(['name'],axis=1)
        vehicleCategory_series = vehicleCategory.stack()
        vehicleCategory_normalized = vehicleCategory_series.to_frame().T
        vehicleCategory_normalized.columns = ['_'.join(col).strip() for col in vehicleCategory_normalized.columns.values]    

        vehicleCategory = vehicleCategory_normalized


        # clean up modes to include in dataframe
        modes = tmp['stats']['modes']
        modes = json_normalize(modes)
        modes = modes.drop(['id'], axis=1)
        modes.index = modes.name
        modes = modes.drop(['name'],axis=1)
        modes_series = modes.stack()
        modes_normalized = modes_series.to_frame().T
        modes_normalized.columns = ['_'.join(col).strip() for col in modes_normalized.columns.values]

        modes = modes_normalized


        os.remove("./tmp.pkl") 

        stats = convert_int(stats)
        modes = convert_int(modes)
        weaponCategory = convert_int(weaponCategory)
        vehicleCategory = convert_int(vehicleCategory)

        stats = fill_missing(stats)
        modes = fill_missing(modes)
        weaponCategory = fill_missing(weaponCategory)
        vehicleCategory = fill_missing(vehicleCategory)


        s = pd.concat([players,stats,modes, weaponCategory, vehicleCategory],axis=1,sort = False)
        try:
            player_info = player_info.append(s,ignore_index=True)
            sd = pd.DataFrame({"player":s['name'],"status":"success"})
            success_count = success_count.append(sd,ignore_index = True)

        except Exception as e:
            sde = pd.DataFrame({"player":s['name'],"status":"failure"})
            success_count = success_count.append(sde,ignore_index = True)
            continue
        pbar.update(1)

 68%|████████████████████████████▋             | 5913/8639 [1:37:25<31:19,  1.45it/s]


In [47]:
success_count.tail()

,player,status
5908,zzRaphaelDz,success
5909,zzola01,success
5910,zztoc,success
5911,zzvw,success
5912,zzzkrak GFYzzz,success


In [48]:
player_info.tail()

,name,avengerKills,bestStreak,deaths,dogtagsTaken,elo,extra.accuracy,extra.assignments,extra.assignmentsTotal,extra.gspm,...,AA 9K22 Tunguska_extra.spm,AA 9K22 Tunguska_stat.destroys,AA 9K22 Tunguska_stat.kills,AA 9K22 Tunguska_stat.time,AA LAV-AD_extra.kpm,AA LAV-AD_extra.spm,AA LAV-AD_stat.destroys,AA LAV-AD_stat.kills,AA LAV-AD_stat.score,AA LAV-AD_stat.time
5908,zzRaphaelDz,23,1,239,2,0,9,0,108,107,...,0,0,0,0,0,0,0,0,0,0
5909,zzola01,1715,1,4985,243,0,15,37,108,306,...,0,0,0,0,0,0,0,0,3007,0
5910,zztoc,196,1,2088,23,0,11,10,108,112,...,0,0,0,0,0,0,0,0,12104,0
5911,zzvw,236,1,2368,54,0,16,12,108,161,...,0,0,0,0,0,0,0,0,15828,0
5912,zzzkrak GFYzzz,1063,0,11440,132,0,7,25,108,115,...,0,0,0,0,0,0,0,0,22879,0


In [50]:
player_info.to_csv('player_info_part5.csv')
success_count.to_csv('success_count_part5.csv')